In [1]:
from skimage.io import imread, imshow, imsave

img_a = imread('image.png')
imshow(img_a)
img_a.min(), img_a.max(), img_a.shape

import numpy as np
r, g, b = np.dsplit(img_a, 3)

sigma = 0.66
half = round(3 * sigma)
half

def mirror(seq, half):
    output = list(seq[::-1][-half:])
    output.extend(seq[:])
    output.extend(seq[-1:-(half + 1):-1])
    return output

def mirroring_border(arr, half):
    arr_mirrored = mirror([mirror(sublist, half) for sublist in arr], half)
    arr_mirrored = np.array(arr_mirrored)
    return arr_mirrored

r_mirrored, g_mirrored, b_mirrored = [mirroring_border(i, half) for i in (r, g, b)]
r_mirrored.shape
img_a_mirrored = np.dstack((r_mirrored, g_mirrored, b_mirrored))
imshow(img_a_mirrored)
imsave('a-mirrored.png', img_a_mirrored)

import math
k = half * 2 + 1
kernel = []
for i in reversed(range(-half, half + 1)):
    for j in range(-half, half + 1):
        kernel.append((j, i))
kernel = (np.array(kernel).reshape(k, k, 2)).tolist()
for i in range(k):
    for j in range(k):
        kernel[i][j] = 1 / (2 * math.pi * sigma ** 2) * math.exp(-(kernel[i][j][0] ** 2 + kernel[i][j][1] ** 2) / (2 * sigma ** 2))
sum = np.array(kernel).sum()
for i in range(k):
    for j in range(k):
        kernel[i][j] = kernel[i][j] / sum

from scipy.signal import convolve2d
r_gauss, g_gauss, b_gauss = [(convolve2d(i[:, :, 0], kernel, mode='valid')).astype('uint8') for i in (r_mirrored, g_mirrored, b_mirrored)]
r_gauss.shape
img_a_gauss = np.dstack((r_gauss, g_gauss, b_gauss))
imshow(img_a_gauss)
imsave('a-gauss.png', img_a_gauss)

FileNotFoundError: No such file: 'c:\Users\stacy\Desktop\cv\image_processing\image.png'

In [ ]:
# --- 5.1 Теорема о свёртке.ipynb ---
from skimage.io import imread, imshow, imsave
img = imread('tiger-gray.png')
imshow(img)
img.shape

import numpy as np
from scipy.signal.windows import gaussian

sigma = 4
n_points = 2 * round(sigma * 3) + 1
kernel = np.outer(gaussian(n_points, std=sigma), gaussian(n_points, std=sigma))
kernel = kernel / kernel.sum()
imshow(kernel, cmap='gray')
kernel.shape, kernel.min(), kernel.max()

gauss = np.zeros((img.shape[0], img.shape[1]))
row = (img.shape[0] - kernel.shape[0]) // 2
col = (img.shape[1] - kernel.shape[1]) // 2
gauss[row:-(img.shape[0]-row-kernel.shape[0]), col:-(img.shape[1]-col-kernel.shape[1])] = kernel[:,:]
imshow(gauss, cmap='gray')
gauss.shape, gauss.min(), gauss.max()

import scipy.fftpack as fp
freq_img = fp.fft2(img)
imshow(np.log(1 + np.abs(fp.fftshift(freq_img))), cmap='gray')
freq_gauss = fp.fft2(fp.ifftshift(gauss))
imshow(np.log(1 + np.abs(fp.fftshift(freq_gauss))), cmap='gray')
multiple = freq_img * freq_gauss
imshow(np.log(1 + np.abs(np.fft.fftshift(multiple))), cmap='gray')
out_img = fp.ifft2(multiple).real
imshow(out_img, cmap='gray')




In [ ]:

# --- 5.1 Алгоритм сжатия изображений JPEG.ipynb ---
%matplotlib inline
import numpy as np

img = np.array([[52, 55, 61, 66, 70, 61, 64, 73],
                [63, 59, 55, 90, 109, 85, 69, 72],
                [62, 59, 68, 113, 144, 104, 66, 73],
                [63, 58, 71, 122, 154, 106, 70, 69],
                [67, 61, 68, 104, 126, 88, 68, 70],
                [79, 65, 60, 70, 77, 68, 58, 75],
                [85, 71, 64, 59, 55, 61, 65, 83],
                [87, 79, 69, 68, 65, 76, 78, 94]])
g = img - 128
g

from math import cos, pi, sqrt
from matplotlib import pyplot as plt

fig, axes = plt.subplots(nrows=8, ncols=8, figsize=(5, 5))
dct = np.zeros((8, 8, 8, 8))
for u in range(8):
    for v in range(8):
        for x in range(8):
            for y in range(8):
                dct[u, v, x, y] = cos((2*x+1)*u*pi/16) * cos((2*y+1)*v*pi/16)
vmin, vmax = dct.min(), dct.max()
for u in range(8):
    for v in range(8):
        axes[u, v].imshow(dct[u, v], cmap='gray', vmin=vmin, vmax=vmax)
        axes[u, v].axis('off')
fig.patch.set_facecolor('#ececec')

a = np.zeros((8, 8))
sgrt_2 = 1 / sqrt(2)
alpha = lambda z: sgrt_2 if z == 0 else 1

for u in range(8):
    for v in range(8):
        auv = 0
        for x in range(8):
            for y in range(8):
                auv += g[x, y] * dct[u, v, x, y]
        a[u, v] = 0.25 * alpha(u) * alpha(v) * auv

for row in a:
    print(' '.join([str(round(s, 2)).rjust(7) for s in row]))

quant = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                  [12, 12, 14, 19, 26, 58, 60, 55],
                  [14, 13, 16, 24, 40, 57, 69, 56],
                  [14, 17, 22, 29, 51, 87, 80, 62],
                  [18, 22, 37, 56, 68, 109, 103, 77],
                  [24, 35, 55, 64, 81, 104, 113, 92],
                  [49, 64, 78, 87, 103, 121, 120, 101],
                  [72, 92, 95, 98, 112, 100, 103, 99]])

quant_a = np.array((a / quant).round(), dtype=int)
quant_a